# Airport Location and Terrain Analysis Demo

This notebook demonstrates DCS APIs for finding airports and checking terrain types.

## Setup

In [1]:
_ = net.lua2json

nil

## Airport Discovery

Get all airbases on the map:

In [2]:
-- Get all airbases on the map
local airbases = world.getAirbases()

-- Extract basic info for first few airbases
local airbase_info = {}
for i = 1, math.min(5, #airbases) do
    local ab = airbases[i]
    airbase_info[i] = {
        name = ab:getName(),
        id = ab:getID(),
        callsign = ab:getCallsign(),
        position = ab:getPoint(),
        category = ab:getDesc().category -- 0=AIRDROME, 1=HELIPAD, 2=SHIP
    }
end

return _(airbase_info)

[{"name":"Rota Intl","callsign":"Rota Intl","id":1,"category":0,"position":{"y":173.38017272949,"x":75950.7734375,"z":47661.18359375}},{"name":"Saipan Intl","callsign":"Saipan Intl","id":2,"category":0,"position":{"y":65.000068664551,"x":179620.453125,"z":100743.234375}},{"name":"Tinian Intl","callsign":"Tinian Intl","id":3,"category":0,"position":{"y":73.154266357422,"x":166672,"z":88785.890625}},{"name":"Antonio B. Won Pat Intl","callsign":"Antonio B. Won Pat Intl","id":4,"category":0,"position":{"y":77.79207611084,"x":-588.52325439453,"z":-1388.0504150391}},{"name":"Olf Orote","callsign":"Olf Orote","id":5,"category":0,"position":{"y":28.506044387817,"x":-4847.6000976562,"z":-16372.65625}}]

## Terrain Type Analysis

Check surface types at different coordinates:

In [3]:
-- Function to convert surface type number to readable text
function getSurfaceTypeName(surfaceType)
    local types = {
        [1] = "LAND",
        [2] = "SHALLOW_WATER", 
        [3] = "WATER",
        [4] = "ROAD",
        [5] = "RUNWAY"
    }
    return types[surfaceType] or "UNKNOWN"
end

-- Test different coordinate types
local test_coords = {
    {x = -100000, z = 500000, name = "Test Point 1"},
    {x = -200000, z = 600000, name = "Test Point 2"},
    {x = 0, z = 0, name = "Origin"}
}

-- Analyze each coordinate
local results = {}
for i, coord in ipairs(test_coords) do
    local surface_type = land.getSurfaceType({x = coord.x, y = coord.z})
    local height = land.getHeight({x = coord.x, y = coord.z})
    
    results[i] = {
        name = coord.name,
        coordinates = {x = coord.x, z = coord.z},
        surface_type = surface_type,
        surface_name = getSurfaceTypeName(surface_type),
        height = height
    }
end

return _(results)

[{"coordinates":{"z":500000,"x":-100000},"name":"Test Point 1","height":0,"surface_type":3,"surface_name":"WATER"},{"coordinates":{"z":600000,"x":-200000},"name":"Test Point 2","height":0,"surface_type":3,"surface_name":"WATER"},{"coordinates":{"z":0,"x":0},"name":"Origin","height":77.79207611084,"surface_type":5,"surface_name":"RUNWAY"}]

## Find Specific Airport

Try to find a specific airport by name:

In [4]:
-- Try to find a specific airport (change name as needed for your map)
-- Common Marianas airports: "Rota Intl", "Saipan Intl", "Tinian Intl", "Andersen AFB"
local airport_name = "Saipan Intl"  -- Change this to an airport on your map
local airport = Airbase.getByName(airport_name)

if airport then
    local info = {
        name = airport:getName(),
        position = airport:getPoint(),
        category = airport:getDesc().category,
        coalition = airport:getCoalition()
    }
    
    -- Check terrain at airport location
    local ap_pos = airport:getPoint()
    local surface_type = land.getSurfaceType({x = ap_pos.x, y = ap_pos.z})
    info.surface_type = surface_type
    info.surface_name = getSurfaceTypeName(surface_type)
    info.height = land.getHeight({x = ap_pos.x, y = ap_pos.z})
    
    return _(info)
else
    return "Airport not found. Try changing the name to match your map."
end

{"coalition":0,"surface_name":"RUNWAY","name":"Saipan Intl","position":{"y":65.000068664551,"x":179620.453125,"z":100743.234375},"height":65.000068664551,"surface_type":5,"category":0}

## Smart Spawn Location Finding

Find a safe land location near the airport:

In [5]:
-- Function to find safe spawn location near airport
function findSafeSpawnLocation(center_x, center_z, search_radius)
    local search_radius = search_radius or 1000
    local step_size = 100
    
    -- Search in a spiral pattern
    for radius = 0, search_radius, step_size do
        for angle = 0, 360, 45 do
            local rad = math.rad(angle)
            local test_x = center_x + radius * math.cos(rad)
            local test_z = center_z + radius * math.sin(rad)
            
            local surface_type = land.getSurfaceType({x = test_x, y = test_z})
            
            -- Check if it's land (not water or runway)
            if surface_type == 1 then  -- LAND
                return {
                    x = test_x,
                    z = test_z,
                    surface_type = surface_type,
                    surface_name = getSurfaceTypeName(surface_type),
                    distance_from_center = radius
                }
            end
        end
    end
    
    return nil  -- No safe location found
end

-- Re-find the airport in case variable wasn't carried over
local airport_name = "Saipan Intl"  -- Change this to match your airport
airport = Airbase.getByName(airport_name)

-- Find a safe spawn location near the airport
if airport then
    local ap_pos = airport:getPoint()
    safe_location = findSafeSpawnLocation(ap_pos.x, ap_pos.z, 2000)
    
    if safe_location then
        -- Add visual marker
        local marker_pos = {x = safe_location.x, y = land.getHeight({x = safe_location.x, y = safe_location.z}), z = safe_location.z}
        trigger.action.smoke(marker_pos, trigger.smokeColor.Green)
        
        trigger.action.outText("Safe spawn location found near " .. airport:getName() .. " (green smoke)", 10)
        
        return _(safe_location)
    else
        return "No safe spawn location found near airport"
    end
else
    return "Airport '" .. airport_name .. "' not found. Try different airport names for Marianas map."
end

{"x":179420.453125,"z":100743.234375,"surface_name":"LAND","surface_type":1,"distance_from_center":200}

## Airport-based Vehicle Spawning

Spawn a vehicle at the validated safe location:

In [6]:
-- Re-find airport and safe location in case variables weren't carried over
local airport_name = "Saipan Intl"  -- Change this to match your airport
airport = Airbase.getByName(airport_name)

if airport then
    local ap_pos = airport:getPoint()
    safe_location = findSafeSpawnLocation(ap_pos.x, ap_pos.z, 2000)
end

-- Spawn a vehicle at the safe location we found
if airport and safe_location then
    local tank_group = {
        name = "Airport_Tank",
        task = "Ground Nothing",
        units = {
            [1] = {
                name = "Airport_Tank_Unit",
                type = "M-1 Abrams",
                x = safe_location.x,
                y = safe_location.z,
                heading = 0,
                skill = "Average"
            }
        }
    }
    
    coalition.addGroup(country.id.USA, Group.Category.GROUND, tank_group)
    
    -- Add blue smoke at airport
    local ap_pos = airport:getPoint()
    trigger.action.smoke(ap_pos, trigger.smokeColor.Blue)
    
    trigger.action.outText("Tank spawned near " .. airport:getName() .. " (blue smoke at airport)", 10)
    
    return "Tank spawned successfully at validated terrain location"
else
    return "Cannot spawn - no safe location or airport available"
end

Tank spawned successfully at validated terrain location

## Cleanup

Remove the spawned vehicle:

In [7]:
-- Clean up spawned units
local group = Group.getByName("Airport_Tank")
if group then
    group:destroy()
    trigger.action.outText("Tank removed", 5)
else
    return "No tank to remove"
end

nil